### Шаг 1. Создание проекта и приложения

#### Создание Django-проекта

1. В терминале создайте новый проект Django:
   ```bash
   django-admin startproject myproject
   cd myproject
   ```

2. Создайте приложение `products`, где будет реализована модель и все представления для CRUD-операций:
   ```bash
   python manage.py startapp products
   ```

3. В `settings.py` добавьте `products` в `INSTALLED_APPS`:
   ```python
   # myproject/settings.py
   INSTALLED_APPS = [
       # стандартные приложения Django
       'products',  # новое приложение
   ]
   ```

### Шаг 2. Настройка структуры приложения `products`

Создайте структуру папок и файлов в `products/` для организации кода:

```bash
myproject/
├── products/
│   ├── migrations/
│   ├── templates/
│   │   └── products/
│   │       ├── product_list.html
│   │       ├── product_detail.html
│   │       ├── product_form.html
│   │       └── product_confirm_delete.html
│   ├── __init__.py
│   ├── admin.py
│   ├── apps.py
│   ├── forms.py
│   ├── models.py
│   ├── tests.py
│   ├── urls.py
│   └── views.py
└── manage.py
```

### Шаг 3. Модель `Product`

В `models.py` создаем модель `Product` с полями для хранения данных о продукте.

```python
# products/models.py
from django.db import models

class Product(models.Model):
    name = models.CharField(max_length=100)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    description = models.TextField(blank=True)
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)

    def __str__(self):
        return self.name
```

После создания модели выполните миграции:
```bash
python manage.py makemigrations
python manage.py migrate
```

### Шаг 4. Форма `ProductForm`

В `forms.py` создаем форму для модели `Product`.

```python
# products/forms.py
from django import forms
from .models import Product

class ProductForm(forms.ModelForm):
    class Meta:
        model = Product
        fields = ['name', 'price', 'description']
```

### Шаг 5. Представления для CRUD

Добавим представления для операций создания, чтения, обновления и удаления в `views.py`.

```python
# products/views.py
from django.shortcuts import render, get_object_or_404, redirect
from .models import Product
from .forms import ProductForm

def product_list(request):
    products = Product.objects.all()
    return render(request, 'products/product_list.html', {'products': products})

def product_detail(request, pk):
    product = get_object_or_404(Product, pk=pk)
    return render(request, 'products/product_detail.html', {'product': product})

def product_create(request):
    if request.method == 'POST':
        form = ProductForm(request.POST)
        if form.is_valid():
            form.save()
            return redirect('product_list')
    else:
        form = ProductForm()
    return render(request, 'products/product_form.html', {'form': form})

def product_update(request, pk):
    product = get_object_or_404(Product, pk=pk)
    if request.method == 'POST':
        form = ProductForm(request.POST, instance=product)
        if form.is_valid():
            form.save()
            return redirect('product_detail', pk=product.pk)
    else:
        form = ProductForm(instance=product)
    return render(request, 'products/product_form.html', {'form': form})

def product_delete(request, pk):
    product = get_object_or_404(Product, pk=pk)
    if request.method == 'POST':
        product.delete()
        return redirect('product_list')
    return render(request, 'products/product_confirm_delete.html', {'product': product})
```

### Шаг 6. URL-маршруты

Создаем маршруты для каждого представления в `urls.py` приложения `products`.

```python
# products/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('', views.product_list, name='product_list'),
    path('product/<int:pk>/', views.product_detail, name='product_detail'),
    path('product/new/', views.product_create, name='product_create'),
    path('product/<int:pk>/edit/', views.product_update, name='product_update'),
    path('product/<int:pk>/delete/', views.product_delete, name='product_delete'),
]
```

Затем включите маршруты приложения `products` в основной файл `urls.py` проекта.

```python
# myproject/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('products/', include('products.urls')),  # подключение маршрутов приложения
]
```

### Шаг 7. Шаблоны

1. **Список продуктов** (`product_list.html`):
    ```html
    <!-- products/templates/products/product_list.html -->
    <h1>Список продуктов</h1>
    <a href="{% url 'product_create' %}">Добавить продукт</a>
    <ul>
        {% for product in products %}
            <li>
                <a href="{% url 'product_detail' product.pk %}">{{ product.name }}</a> - ${{ product.price }}
            </li>
        {% endfor %}
    </ul>
    ```

2. **Детали продукта** (`product_detail.html`):
    ```html
    <!-- products/templates/products/product_detail.html -->
    <h1>{{ product.name }}</h1>
    <p>Цена: ${{ product.price }}</p>
    <p>Описание: {{ product.description }}</p>
    <a href="{% url 'product_edit' product.pk %}">Редактировать</a>
    <a href="{% url 'product_delete' product.pk %}">Удалить</a>
    <a href="{% url 'product_list' %}">Назад к списку</a>
    ```

3. **Форма для создания и редактирования** (`product_form.html`):
    ```html
    <!-- products/templates/products/product_form.html -->
    <h1>{% if form.instance.pk %}Редактировать{% else %}Добавить{% endif %} продукт</h1>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">Сохранить</button>
    </form>
    <a href="{% url 'product_list' %}">Назад к списку</a>
    ```

4. **Подтверждение удаления** (`product_confirm_delete.html`):
    ```html
    <!-- products/templates/products/product_confirm_delete.html -->
    <h1>Удалить {{ product.name }}?</h1>
    <p>Вы уверены, что хотите удалить этот продукт?</p>
    <form method="post">
        {% csrf_token %}
        <button type="submit">Да, удалить</button>
        <a href="{% url 'product_detail' product.pk %}">Отмена</a>
    </form>
    ```

### Шаг 8. Тестирование проекта

Запустите сервер и проверьте работу CRUD-операций:

```bash
python manage.py runserver
```